# Homework 1. - Trevor Grant - Github: tjgran01

For this proect I tried to be too fancy and predict a Pokemon's Type, based solely on the image of the pokemon. It did not work. How sad. I did, however, get tensorflow running on my GPU and learned some data cleaning stuff in the process, so that felt pretty cool. I also combined two datasets. I still failed though. Which, that's pretty much life.

### Data:

Data was gathered from two datasets on Kaggle.com. The links are below:

imgs:https://www.kaggle.com/hannesrosenbusch/6036-labeled-pokemon-pictures

labels: https://www.kaggle.com/abcsds/pokemon



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import os

from sklearn.model_selection import train_test_split

In [2]:
# Keras Imports
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.optimizers import SGD, RMSprop, Adam

Using TensorFlow backend.


### Running on the GPU

I learned a lot about how to make the model run on the GPU, but was still having trouble when push came to shove. I was glad to see that I was note alone and found a somewhat helpful (in that it made the model run) thread on github.com 

https://github.com/tensorflow/tensorflow/issues/24496

This is apparently an issue with the newer NVIDA card running on Ubuntu --- so if anyone ran into this issue the below code should fix the problem.

In [3]:
# So Tensorflow runs on GPU
import tensorflow as tf

# For running on GPU.
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

### Param Dictionary

I put the parameters for the model into a dictionary because lose variables make me uncomfortable.

In [4]:
def get_param_dict():

    # From "Deeplearning with Keras"
    EPOCHS = 50
    BATCH_SIZE = 16
    VERBOSE = 1
    OPTIMIZER = Adam()
    VALIDATION_SPLIT=0.15
    IMG_ROWS, IMG_COLS = 100, 100 # input image dimensions
    CLASSES = 4 # number of outputs = number of digits
    INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 3)

    param_dict = {"EPOCHS": EPOCHS, "BATCH_SIZE": BATCH_SIZE,
                  "VERBOSE": VERBOSE, "OPTIMIZER": Adam(),
                  "VALIDATION_SPLIT": VALIDATION_SPLIT, "IMG_ROWS": IMG_ROWS,
                  "IMG_COLS": IMG_COLS, "CLASSES": CLASSES,
                  "INPUT_SHAPE": INPUT_SHAPE}

    return param_dict

### Cleaning the data

This probably should have been multiple functions, but oh well. This was the bulk of my work so I could better understand what format data needed to be in in order to be fed into the model. Reading the book, he often just pulls datasets from the keras API. I find that when I see that it is difficult for me to learn exactly **what** is going into the model, so being able to put together a dataset somewhat assuaged my concerns there.

### Downsampling.

Looking over the label distribution I had a major problem in that many of the Pokemon were of 'water' type (i.e. it was the majority class by a long shot with about 20% more samples than the next most common class). I added some functionality in here that would allow me to drop samples from types of pokemon I didn't want to classify. I thought that that might work --- but it didn't --- still a neat way to do it though!

In total there were 18 different classes. When I ran the model on all of the classes it easily obtained a score of 97% accuracy ... on the validation set ... and ~18% on the test set. Boo. Downsampling to a binary classification problem helped with test accuracy somewhat, but not by much, but the validation set took a plunge.

In [5]:
def relabel_by_type():


    # Adjust drop list to get rid of minority classes, etc.
    drop_list = ["drop_me"]
    drop_list = ["drop_me", 'Bug', 'Dark', 'Dragon', 'Electric', 'Fairy',
                 'Fighting', 'Flying', 'Ghost', 'Ground', 'Normal', 'Psychic', 'Rock', 'Steel']

    # Get Type Labels For All Images.

    imgs = np.load(f"{os.getcwd()}/../data/poke_image_data.npy")
    names_and_scores = pd.read_csv(f"{os.getcwd()}/../data/names_and_strengths.csv")
    names_and_stats = pd.read_csv(f"{os.getcwd()}/../data/Pokemon.csv")

    type_list = []
    type_2_list = []
    for name in names_and_scores["name"].tolist():
        if name in names_and_stats["Name"].tolist():
            type_list.append(names_and_stats.loc[names_and_stats["Name"] == name, "Type 1"].iloc[0])
            type_2_list.append(names_and_stats.loc[names_and_stats["Name"] == name, "Type 2"].iloc[0])
        else:
            type_list.append("drop_me")
            type_2_list.append("drop_me")


    names_and_scores["Type 1"] = type_list
    names_and_scores["Type 2"] = type_2_list

    entries_to_drop = []
    for i, row in (names_and_scores.iterrows()):
        if row["Type 1"] in drop_list or row["Type 2"] in drop_list:
            entries_to_drop.append(i)

    poke_data = names_and_scores.drop(entries_to_drop, axis=0)
    poke_imgs = []
    for i, img in enumerate(imgs):
        if i not in entries_to_drop:
            poke_imgs.append(imgs[i])

    poke_imgs = np.array(poke_imgs)

    poke_data.to_csv(f"{os.getcwd()}/../data/relabeled/poke_data.csv")
    np.save(f"{os.getcwd()}/../data/relabeled/poke_imgs.npy", poke_imgs)


### Plotting

It was important to be able to plot an image by pokemon name to ensure that my data cleaning didn't shuffle things around and mislabel things. That is all this function is for, really.

In [6]:
def plot_sample_pokemon(pokename, imgs, df):
    """Ensures Re-Labeling Worked By Printing out A Pokemon img, given a name."""

    df = df[df["name"] == pokename]
    picked = random.choice(df.index.tolist())

    imgplot = plt.imshow(imgs[picked])
    plt.show()

### One Hot Encoding

This is my poor-mans way of implementing one hot encoding. I know there are functions for this sort of thing, but based on the way the data was formatted this seemed to make the most sense as it preserved the labeling structure so that the indexes of the dataframe didn't come off from the indexes of the image (as they were sourced from different data sets).

In [7]:
def one_hot_labels(df):

    types = ["Grass", "Fire", "Ice", "Poison"]

    one_hot = pd.get_dummies(df['Type 1'])
    df = df.join(one_hot)

    df["one_hot"] = df[types].values.tolist()

    return df

In [8]:
def organize_train_test_sets(df, imgs):

    imgs = imgs.astype("float32")

    labels = df["one_hot"].tolist()
    labels = np.array(labels)
    labels = labels.astype("float32")

    return train_test_split(imgs, labels, shuffle=True, test_size=0.15)

In [9]:
def make_model(X_train, X_test, y_train, y_test, params):

    model = Sequential()

    model.add(Conv2D(16, kernel_size=5, input_shape=params["INPUT_SHAPE"]))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, kernel_size=5))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(20, 20)))

    model.add(Flatten())
    model.add(Dense(50))
    model.add(Activation("relu"))

    model.add(Dense(params["CLASSES"]))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy",
                  optimizer='rmsprop',
                  metrics=["accuracy"])
    model.summary()

    history = model.fit(X_train, y_train,batch_size=params["BATCH_SIZE"],
                        epochs=params["EPOCHS"], verbose=1,
                        validation_split=params["VALIDATION_SPLIT"])
    score = model.evaluate(X_test, y_test, verbose=1)
    print(score)


In [10]:
def main():

    params = get_param_dict()

    if not os.path.exists(f"{os.getcwd()}/../data/relabeled/poke_data.csv"):
        relabel_by_type()

    poke_imgs = np.load(f"{os.getcwd()}/../data/relabeled/poke_imgs.npy")
    poke_data = pd.read_csv(f"{os.getcwd()}/../data/relabeled/poke_data.csv")

    plot_sample_pokemon("Pichu", poke_imgs, poke_data)

    # Get one hot encoding for each type.
    poke_data = one_hot_labels(poke_data)
    # Split train and test sets
    X_train, X_test, y_train, y_test = organize_train_test_sets(poke_data, poke_imgs)

    model = make_model(X_train, X_test, y_train, y_test, params)

In [12]:
main()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 96, 96, 16)        1216      
_________________________________________________________________
activation_5 (Activation)    (None, 96, 96, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 44, 44, 32)        12832     
_________________________________________________________________
activation_6 (Activation)    (None, 44, 44, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)              

953/953 [==============================] - 0s 265us/step - loss: 54863983.7524 - accuracy: 0.3190 - val_loss: 74009133.6331 - val_accuracy: 0.1124
Epoch 42/50
953/953 [==============================] - 0s 269us/step - loss: 61506726.5477 - accuracy: 0.3347 - val_loss: 97546118.6746 - val_accuracy: 0.6627
Epoch 43/50
953/953 [==============================] - 0s 321us/step - loss: 59980414.2455 - accuracy: 0.3295 - val_loss: 41099288.0947 - val_accuracy: 0.2071
Epoch 44/50
953/953 [==============================] - 0s 324us/step - loss: 68090487.8153 - accuracy: 0.3326 - val_loss: 86051022.1538 - val_accuracy: 0.1953
Epoch 45/50
953/953 [==============================] - 0s 314us/step - loss: 69679170.6359 - accuracy: 0.3410 - val_loss: 59052904.3314 - val_accuracy: 0.1953
Epoch 46/50
953/953 [==============================] - 0s 308us/step - loss: 83368832.1280 - accuracy: 0.3169 - val_loss: 38022271.1953 - val_accuracy: 0.6272
Epoch 47/50
953/953 [==============================] - 0s 